### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

/home/randomwalk10/anaconda3/envs/aind/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [4]:
# cell for work
print("The number of movies in the dataset:", movies.movie_id.nunique())

The number of movies in the dataset: 32920


In [5]:
# cell for work
print("The number of ratings in the dataset:", reviews.rating.count())

The number of ratings in the dataset: 762277


In [6]:
# more work
set_genre = set()
for x in movies.genre.str.split("|").tolist():
    if x is not np.nan:
        for y in x:
            set_genre.add(y)
            
print("The number of different genres:", len(set_genre))

The number of different genres: 28


In [7]:
# and still more
print("The number of unique users in the dataset:", reviews.user_id.nunique())

The number of unique users in the dataset: 56390


In [8]:
# and even more - open additional cells as necessary
print("The number missing ratings in the reviews dataset:", reviews.rating.isnull().sum())

The number missing ratings in the reviews dataset: 0


In [9]:
print("The average rating given across all ratings:", reviews.rating.mean())

The average rating given across all ratings: 7.298336431507182


In [10]:
print("The minimum rating given across all ratings:", reviews.rating.min())

The minimum rating given across all ratings: 0


In [11]:
print("The minimum rating given across all ratings:", reviews.rating.max())

The minimum rating given across all ratings: 10


In [12]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 32907,
'The number of ratings in the dataset': 761910,
'The number of different genres': 28,
'The number of unique users in the dataset': 56380,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.298316074076991,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [13]:
# dates of movies
movies['date'] = movies['movie'].apply(
    lambda x: int(x[x.find("(")+1:x.find(")")]))
movies['1800''s'] = movies['movie'].apply(
    lambda x: 1 if int(x[x.find("(")+1:x.find(")")])-1800 in range(100) else 0)
movies['1900''s'] = movies['movie'].apply(
    lambda x: 1 if int(x[x.find("(")+1:x.find(")")])-1900 in range(100) else 0)
movies['2000''s'] = movies['movie'].apply(
    lambda x: 1 if int(x[x.find("(")+1:x.find(")")])-2000 in range(100) else 0)

In [14]:
# genres of movies
for label in list(set_genre):
    movies[label] = movies.genre.apply(
    lambda x: 1 if x is not np.nan and label in x.split('|') else 0)

In [15]:
# head of movies
movies.head()

,movie_id,movie,genre,date,1800s,1900s,2000s,Adventure,Sport,Game-Show,...,Adult,Drama,Music,News,Thriller,Reality-TV,Fantasy,Animation,Film-Noir,History
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# reviews
reviews['date'] = pd.to_datetime(reviews['timestamp'],unit='s')

In [17]:
# months
pd_months = pd.DatetimeIndex(reviews['date']).month
list_months = np.unique(pd_months.tolist())
list_months.sort()
for month in list_months:
    label = "month_"+str(month)
    reviews[label] = pd_months
    reviews[label] = reviews[label].apply(lambda x: 1 if x==month else 0)

In [18]:
# years
pd_years = pd.DatetimeIndex(reviews['date']).year
list_years = np.unique(pd_years.tolist())
list_years.sort()
for year in list_years:
    label = "year_"+str(year)
    reviews[label] = pd_years
    reviews[label] = reviews[label].apply(lambda x: 1 if x==year else 0)

In [19]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019
0,1,0111161,10,1373234211,2013-07-07 21:56:51,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0117060,7,1373415231,2013-07-10 00:13:51,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0120755,6,1373424360,2013-07-10 02:46:00,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,0317919,6,1373495763,2013-07-10 22:36:03,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0454876,10,1373621125,2013-07-12 09:25:25,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [20]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year